In [1]:
from snowflake.snowpark import Session
from configparser import ConfigParser
import snowflake.snowpark.functions as fn
import pandas as pd

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

# Primer analisis

In [3]:
from snowflake.snowpark.functions import col, concat, count_distinct, to_date

from DP_transacciones_olo import get_transacciones_olo

from DP_transacciones_cloud import get_transacciones_cloud

from DP_datos_tiendas import get_datos_tiendas_dp

descuentos = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(col('FCIDDESCUENTO').isin([4493, 4494]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(col('FIIDCLIENTE').is_not_null())
    .filter(col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', concat(col('FCIDTIENDA'), col('FIIDCLIENTE')))
    .with_column('FECHA', to_date('FDIDDIA'))
    .join(descuentos, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', concat(col('FIIDTIENDA'), col('FIIDCLIENTE')))
    .join(ordenes, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo = (
    transacciones_olo
    .join(clientes, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'STORE_ID', 'FECHA'])
)

redenciones_cloud = get_transacciones_cloud(session).filter(col('COUPONSCODE').like('%HELLO%')).select(['EMAIL', 'STORE_ID', 'FECHA'])

redenciones_totales = redenciones_cloud.union_all(redenciones_olo)

datos_tiendas_dp = get_datos_tiendas_dp(session)

tiempo = session.table('WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME')

redenciones_por_region_y_semana = (
    redenciones_totales
    .join(datos_tiendas_dp, on = 'STORE_ID')
    .join(tiempo, on = 'FECHA')
    .group_by(['REGION', 'ANIO_ALSEA', 'SEM_ALSEA'])
    .agg(count_distinct(col('EMAIL')))
    .order_by(['ANIO_ALSEA', 'SEM_ALSEA', 'REGION'])
)

redenciones_por_region_y_semana.to_pandas().to_csv('redenciones_por_region_y_semana.csv', index = False)

-------------------------------------------------------------------------------------------------
|"CC"   |"Nombre"                    |"Región Operativa"  |"Gerente de Distrito"                |
-------------------------------------------------------------------------------------------------
|11252  |San Miguel De Allende       |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11639  |Plaza Luciernaga            |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11942  |Antea                       |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11590  |Bernardo Quintana           |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11635  |Ciudad Del Sol              |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11302  |Jurica                      |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11439  |Queretaro Pie De La Cuesta  |Bajío               |Fernando Alejandro Gutierrez Garcia  |
|11486  |Juriquilla 

In [ ]:
from snowflake.snowpark.functions import mode, count_distinct, col
from DP_transacciones_total import get_transacciones_dp_total
from DP_datos_tiendas import get_datos_tiendas_dp
from DP_registros import get_registros_dp

transacciones_dp_total = get_transacciones_dp_total(session)

datos_tiendas_dp = get_datos_tiendas_dp(session)

usuarios_por_region = transacciones_dp_total.join(datos_tiendas_dp, on = 'STORE_ID').group_by('EMAIL').agg(mode('REGION').alias('REGION'))

registros_dp = get_registros_dp(session)

tiempo = session.table('WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME')

(
    registros_dp
    .join(tiempo, on = 'FECHA')
    .join(usuarios_por_region, on = 'EMAIL')
    .group_by(['REGION', 'ANIO_ALSEA', 'SEM_ALSEA'])
    .agg(count_distinct(col('EMAIL')))
    .order_by(['ANIO_ALSEA', 'SEM_ALSEA', 'REGION'])
).to_pandas().to_csv('registros_por_region_y_semana.csv', index = False)

# Segundo analisis

In [2]:
from DP_transacciones_total import get_transacciones_dp_total
from DP_registros import get_registros_dp
from general_get_tiempo import get_tiempo
import snowflake.snowpark.functions as fn

tiempo = get_tiempo(session)

transacciones_dp_total = get_transacciones_dp_total(session)

registros_dp = get_registros_dp(session)

regiones_dominos = (
    session
    .create_dataframe(pd.read_excel('./regiones_dominos.xlsx'))
    .with_column_renamed('CC', 'STORE_ID')
    .with_column_renamed('Región Operativa', 'REGION')
)

transacciones_dp_total = get_transacciones_dp_total(session)

usuarios_por_region = (
    transacciones_dp_total
    .join(regiones_dominos, on = 'STORE_ID')
    .group_by('EMAIL')
    .agg(fn.mode('REGION').alias('REGION'))
)

registros_dp = get_registros_dp(session)

tiempo = session.table('WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME')

usuarios_registrados_por_region = (
    registros_dp
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('SEM_ALSEA') >= 27)
    .join(usuarios_por_region, on = 'EMAIL')
)

usuarios_registrados_por_region.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
registros_por_region = (
    usuarios_registrados_por_region
    .join(transacciones_dp_total, on = 'EMAIL')
    .group_by('REGION')
    .agg(fn.count_distinct(fn.col('EMAIL')).alias('USUARIOS'), fn.sum('VENTA').alias('VENTA'), fn.count_distinct('ORDER_ID').alias('ORDENES'))
    .with_column('TICKET_PROMEDIO', fn.col('VENTA') / fn.col('ORDENES'))
    .with_column_renamed('USUARIOS', 'REGISTROS')
    .select('REGION', 'REGISTROS', 'ORDENES', 'TICKET_PROMEDIO')
)

registros_por_region.to_pandas().to_excel('registros_por_region.xlsx')

registros_por_region.show()

-----------------------------------------------------------------
|"REGION"        |"REGISTROS"  |"ORDENES"  |"TICKET_PROMEDIO"   |
-----------------------------------------------------------------
|Centro Sur      |29827        |98323      |267.2265267852227   |
|Veracruz        |9331         |30867      |229.80470578443894  |
|Bajío           |5725         |19214      |256.3829804417038   |
|Sureste         |17922        |62840      |240.78602128558578  |
|Occidente       |17173        |54496      |243.8187649330796   |
|Norte-Noroeste  |11971        |40370      |234.49968118182673  |
|Centro Norte    |31673        |103229     |265.227068810188    |
|Sur Oriente     |14149        |41071      |242.9140663896583   |
-----------------------------------------------------------------



In [7]:
venta_por_region = (
    transacciones_dp_total
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('SEM_ALSEA') >= 27)
    .join(regiones_dominos, on = 'STORE_ID')
    .group_by('REGION')
    .agg(fn.sum('VENTA').alias('VENTA'), fn.count_distinct('ORDER_ID').alias('ORDENES'))
    .with_column('TICKET_PROMEDIO', fn.col('VENTA') / fn.col('ORDENES'))
)

venta_por_region.to_pandas().to_excel('venta_por_region.xlsx')

venta_por_region.show()

------------------------------------------------------------------------
|"REGION"        |"VENTA"             |"ORDENES"  |"TICKET_PROMEDIO"   |
------------------------------------------------------------------------
|Centro Sur      |151836718.96551722  |547517     |277.3187297664131   |
|Bajío           |26000033.534482762  |101122     |257.1154994410985   |
|Veracruz        |38974137.155172415  |167307     |232.94982968538324  |
|Centro Norte    |154244827.19827586  |560097     |275.3894900316836   |
|Norte-Noroeste  |53246351.07758621   |219431     |242.65646639529606  |
|Sur Oriente     |56458360.344827585  |226103     |249.70195152133135  |
|Sureste         |92639450.0          |378732     |244.60423201630704  |
|Occidente       |76058593.75         |307395     |247.42950845003986  |
------------------------------------------------------------------------



In [5]:
from DP_transacciones_olo import get_transacciones_olo
from DP_transacciones_cloud import get_transacciones_cloud

descuentos_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4493, 4494]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_40_OFF, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_40_OFF, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_40_OFF = (
    transacciones_olo
    .join(clientes_40_OFF, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_40_OFF = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like('%HELLO%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_40_OFF = redenciones_cloud_40_OFF.union_all(redenciones_olo_40_OFF)

registros_40_OFF_por_region = (
    usuarios_registrados_por_region
    .join(redenciones_totales_40_OFF, on = 'EMAIL')
    .select('EMAIL', 'REGION')
    .join(transacciones_dp_total, on = 'EMAIL')
    .distinct()
    .group_by('REGION')
    .agg(fn.count_distinct(fn.col('EMAIL')).alias('USUARIOS'), fn.sum('VENTA').alias('VENTA'), fn.count_distinct('ORDER_ID').alias('ORDENES'))
    .with_column('TICKET_PROMEDIO', fn.col('VENTA') / fn.col('ORDENES'))
    .with_column_renamed('USUARIOS', 'REGISTROS')
    .select('REGION', 'REGISTROS', 'ORDENES', 'TICKET_PROMEDIO')
)

registros_40_OFF_por_region.to_pandas().to_excel('registros_40_off.xlsx')

registros_40_OFF_por_region.show()

-----------------------------------------------------------------
|"REGION"        |"REGISTROS"  |"ORDENES"  |"TICKET_PROMEDIO"   |
-----------------------------------------------------------------
|Centro Norte    |6732         |13575      |244.37789737727823  |
|Norte-Noroeste  |2447         |4899       |221.89877772381416  |
|Sur Oriente     |3773         |7018       |223.98227218679062  |
|Centro Sur      |6371         |12578      |240.4193967573377   |
|Bajío           |1124         |2367       |227.99768366767188  |
|Veracruz        |2364         |4494       |212.3491379310345   |
|Sureste         |4102         |8368       |217.81451918639152  |
|Occidente       |3466         |6987       |231.82940115386705  |
-----------------------------------------------------------------



In [6]:
from DP_transacciones_olo import get_transacciones_olo
from DP_transacciones_cloud import get_transacciones_cloud

descuentos_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4647, 4648]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_40_OFF, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_40_OFF, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_GO199 = (
    transacciones_olo
    .join(clientes_GO199, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_GO199 = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like(r'%GO199%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_GO199 = redenciones_cloud_GO199.union_all(redenciones_olo_GO199)

registros_GO199_por_region = (
    usuarios_registrados_por_region
    .join(redenciones_totales_GO199, on = 'EMAIL')
    .select('EMAIL', 'REGION')
    .distinct()
    .join(transacciones_dp_total, on = 'EMAIL')
    .group_by('REGION')
    .agg(fn.count_distinct(fn.col('EMAIL')).alias('USUARIOS'), fn.sum('VENTA').alias('VENTA'), fn.count_distinct('ORDER_ID').alias('ORDENES'))
    .with_column('TICKET_PROMEDIO', fn.col('VENTA') / fn.col('ORDENES'))
    .with_column_renamed('USUARIOS', 'REGISTROS')
    .select('REGION', 'REGISTROS', 'ORDENES', 'TICKET_PROMEDIO')
)

registros_GO199_por_region.to_pandas().to_excel('registros_GO199.xlsx')

registros_GO199_por_region.show()

-----------------------------------------------------------------
|"REGION"        |"REGISTROS"  |"ORDENES"  |"TICKET_PROMEDIO"   |
-----------------------------------------------------------------
|Centro Sur      |5137         |20093      |236.15512221617755  |
|Bajío           |899          |3783       |227.00978515500384  |
|Veracruz        |1705         |5590       |212.08374714699895  |
|Centro Norte    |5397         |20001      |241.3323665713266   |
|Sur Oriente     |2639         |8323       |219.25129574465439  |
|Norte-Noroeste  |1712         |6180       |214.0462769222185   |
|Sureste         |3264         |12576      |217.0130640190401   |
|Occidente       |2575         |9100       |220.63487116331945  |
-----------------------------------------------------------------

